# Modelo BIOS:

## Importacion de Librerias

In [1]:
import pandas as pd
from datetime import datetime, timedelta
import pulp as pu
from problema.asignador_capacidad import AsignadorCapacidad
import os

## Parametros generales

In [2]:
file = 'model_template_1205v2.xlsm'

# Capacidad de carga de un camion
cap_camion = 34000

# Capacidad de descarga en puerto por día
cap_descarge = 5000000

# Costo de backorder por dia
costo_backorder_dia = 250000

## Lectura de dataframes

In [3]:
# Leer el archivo de excel
productos_df = pd.read_excel(io=file, sheet_name='ingredientes')
plantas_df = pd.read_excel(io=file, sheet_name='plantas')
asignador = AsignadorCapacidad(file)
unidades_almacenamiento_df = asignador.obtener_unidades_almacenamiento()
consumo_proyectado_df = pd.read_excel(io=file, sheet_name='consumo_proyectado')
transitos_df = pd.read_excel(io=file, sheet_name='tto_puerto')
inventario_puerto_df = pd.read_excel(io=file, sheet_name='inventario_puerto')
costos_almacenamiento_df = pd.read_excel(io=file, sheet_name='costos_almacenamiento_cargas')
operaciones_portuarias_df = pd.read_excel(io=file, sheet_name='costos_operacion_portuaria')
operaciones_portuarias_df = operaciones_portuarias_df.set_index(['tipo_operacion', 'operador', 'puerto', 'ingrediente'])
fletes_df = pd.read_excel(io=file, sheet_name='fletes_cop_per_kg')
intercompany_df = pd.read_excel(io=file, sheet_name='venta_entre_empresas')

## Creacion de parametros del problema
### Tiempo

In [4]:
# Obtener el conjunto de periodos
fechas = [datetime.strptime(x, '%d/%m/%Y') for x in consumo_proyectado_df.drop(columns=['planta', 'ingrediente']).columns]

periodos = [int(x.strftime('%Y%m%d')) for x in fechas]

periodo_anterior = fechas[0] - timedelta(days=1)
periodo_anterior = int(periodo_anterior.strftime('%Y%m%d'))

print(periodos[0],periodos[-1])

20231130 20231230


### Productos

In [5]:
productos = [productos_df.loc[i]['nombre'] for i in productos_df.index]

### Plantas
#### Tiempo de descarge de materiales

In [6]:
# Generar plantas
plantas = dict()

for j in plantas_df.index:
    planta = plantas_df.loc[j]['planta']
    empresa = plantas_df.loc[j]['empresa']
    operacion_minutos = plantas_df.loc[j]['operacion_minutos']*plantas_df.loc[j]['plataformas']
    plantas[planta] = dict()
    plantas[planta]['empresa'] = empresa
    plantas[planta]['tiempo_total'] = operacion_minutos
    plantas[planta]['tiempo_ingrediente'] = dict()
    plantas[planta]['llegadas_puerto'] = dict()

    for p in productos:
        t_ingrediente = plantas_df.loc[j][p]
        plantas[planta]['tiempo_ingrediente'][p] = t_ingrediente
        plantas[planta]['llegadas_puerto'][p] = {t:list() for t in periodos}

#### Inventario en Planta

In [7]:
unidades_almacenamiento_df['capacidad'] = unidades_almacenamiento_df.apply(lambda x: x[x['ingrediente_actual']] ,axis=1)
unidades_almacenamiento_df.drop(columns=productos, inplace=True)
unidades_almacenamiento_df = unidades_almacenamiento_df.groupby(['planta', 'ingrediente_actual'])[['cantidad_actual', 'capacidad']].sum().reset_index()
# Generar un diccionario para renombrar las columnas de tiempo en consumo proyectado
consumo_proyectado_renamer = {x:datetime.strptime(x, '%d/%m/%Y').strftime('%Y%m%d') for x in consumo_proyectado_df.drop(columns=['planta', 'ingrediente']).columns}
# Efectuar el cambio de nombre
consumo_proyectado_df.rename(columns=consumo_proyectado_renamer, inplace=True)
# Unir con el consumo proyectado
unidades_almacenamiento_df = pd.merge(left=unidades_almacenamiento_df,
                                 right=consumo_proyectado_df,
                                 left_on=['planta', 'ingrediente_actual'],
                                 right_on=['planta', 'ingrediente'],
                                 how='left').drop(columns=['ingrediente']).rename(columns={'ingrediente_actual': 'ingrediente', 'cantidad_actual':'cantidad'}).fillna(0.0)

print(unidades_almacenamiento_df.shape)
unidades_almacenamiento_df.head()

(101, 35)


,planta,ingrediente,cantidad,capacidad,20231130,20231201,20231202,20231203,20231204,20231205,...,20231221,20231222,20231223,20231224,20231225,20231226,20231227,20231228,20231229,20231230
0,barbosa,frijol,123619.2,204000.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,barbosa,gluten,1.0,40000.0,253.183667,253.183667,253.183667,253.183667,253.183667,253.183667,...,253.183667,253.183667,253.183667,253.183667,253.183667,253.183667,253.183667,253.183667,253.183667,253.183667
2,barbosa,maiz,388388.6,408000.0,141663.030000,141663.030000,141663.030000,141663.030000,141663.030000,141663.030000,...,141663.030000,141663.030000,141663.030000,141663.030000,141663.030000,141663.030000,141663.030000,141663.030000,141663.030000,141663.030000
3,barbosa,tsoya,117787.0,204000.0,45964.794000,45964.794000,45964.794000,45964.794000,45964.794000,45964.794000,...,45964.794000,45964.794000,45964.794000,45964.794000,45964.794000,45964.794000,45964.794000,45964.794000,45964.794000,45964.794000
4,bmanga,cascarilla,1.0,34000.0,1378.505000,1378.505000,1378.505000,1378.505000,1378.505000,1378.505000,...,1378.505000,1378.505000,1378.505000,1378.505000,1378.505000,1378.505000,1378.505000,1378.505000,1378.505000,1378.505000


In [8]:
# Llenar la informacion de los inventarios
for i in unidades_almacenamiento_df.index:
    planta = unidades_almacenamiento_df.loc[i]['planta']
    ingrediente = unidades_almacenamiento_df.loc[i]['ingrediente']
    cantidad_inicial = unidades_almacenamiento_df.loc[i]['cantidad']
    capacidad_almacenamiento = unidades_almacenamiento_df.loc[i]['capacidad']

    if not 'inventarios' in plantas[planta].keys():
        plantas[planta]['inventarios'] = dict()

    if not ingrediente in plantas[planta]['inventarios'].keys():
        plantas[planta]['inventarios'][ingrediente] = dict()

    if not 'capacidad' in plantas[planta]['inventarios'][ingrediente].keys():
        plantas[planta]['inventarios'][ingrediente]['capacidad'] = capacidad_almacenamiento

    if not 'inventario_final' in plantas[planta]['inventarios'][ingrediente].keys():
        plantas[planta]['inventarios'][ingrediente]['inventario_final'] = dict()

    if not 'llegadas' in plantas[planta]['inventarios'][ingrediente].keys():
        plantas[planta]['inventarios'][ingrediente]['llegadas'] = dict()

    if not 'consumo' in plantas[planta]['inventarios'][ingrediente].keys():
        plantas[planta]['inventarios'][ingrediente]['consumo'] = dict()

    if not 'backorder' in plantas[planta]['inventarios'][ingrediente].keys():
        plantas[planta]['inventarios'][ingrediente]['backorder'] = dict()

    plantas[planta]['inventarios'][ingrediente]['inventario_final'][periodo_anterior] = cantidad_inicial

    consumo_total = sum([unidades_almacenamiento_df.loc[i][str(t)] for t in periodos])

    if consumo_total > 0:
        for periodo in periodos:
            # Agregar las variables de inventario
            transporte_var_name = f'I_{planta}_{ingrediente}_{periodo}'
            transporte_var = pu.LpVariable(name=transporte_var_name, lowBound=0.0, cat=pu.LpContinuous)
            plantas[planta]['inventarios'][ingrediente]['inventario_final'][periodo] = transporte_var  

            # Agregar las listas a donde llegarán los transportes
            plantas[planta]['inventarios'][ingrediente]['llegadas'][periodo] = list()

            # Agregar las variables de backorder
            bak_var_name = f'B_{planta}_{ingrediente}_{periodo}'
            bak_var = pu.LpVariable(name=bak_var_name, lowBound=0.0, cat=pu.LpContinuous)
            plantas[planta]['inventarios'][ingrediente]['backorder'][periodo] = bak_var

            # Agregar el consumo proyectado
            consumo = unidades_almacenamiento_df.loc[i][str(periodo)]
            plantas[planta]['inventarios'][ingrediente]['consumo'][periodo] = consumo
    else:
        for periodo in periodos:
            # Dejar el inventario en el estado actual
            plantas[planta]['inventarios'][ingrediente]['inventario_final'][periodo] = cantidad_inicial  

            # Agregar el consumo proyectado
            consumo = unidades_almacenamiento_df.loc[i][str(periodo)]
            plantas[planta]['inventarios'][ingrediente]['consumo'][periodo] = consumo


In [9]:
plantas['envigado']['inventarios'].keys()

dict_keys(['cascarilla', 'destilado', 'destiladohp', 'forraje', 'frijol', 'gluten', 'maiz', 'tgirasol', 'trigo', 'tsoya'])

### Cargas en Puerto

In [10]:
# Generar Cargas
cargas = dict()

# A partir de los transitos
for i in transitos_df.index:
    importacion = str(transitos_df.loc[i]['importacion']).replace(' ','')
    empresa = transitos_df.loc[i]['empresa']
    operador = transitos_df.loc[i]['operador']
    puerto = transitos_df.loc[i]['puerto']
    ingrediente = transitos_df.loc[i]['ingrediente']
    cantidad_kg = transitos_df.loc[i]['cantidad_kg']
    valor_cif = transitos_df.loc[i]['valor_kg']
    fecha = transitos_df.loc[i]['fecha_llegada']
    if not importacion in cargas.keys():
        cargas[importacion] = dict()
    
    cargas[importacion]['empresa'] = empresa
    cargas[importacion]['operador'] = operador
    cargas[importacion]['puerto'] = puerto
    cargas[importacion]['ingrediente'] = ingrediente
    cargas[importacion]['valor_cif'] = valor_cif
    cargas[importacion]['inventario_inicial'] = 0
    cargas[importacion]['costo_almacenamiento'] = {int(t.strftime('%Y%m%d')):0 for t in fechas}
    cargas[importacion]['llegadas'] = dict()
    cargas[importacion]['fecha_inicial'] = int(fecha.strftime('%Y%m%d'))
    
    # Poner llegadas de materia
    while cantidad_kg > cap_descarge:
        cargas[importacion]['llegadas'][int(fecha.strftime('%Y%m%d'))] = cap_descarge
        cantidad_kg -= cap_descarge
        fecha = fecha + timedelta(days=1)
    
    if cantidad_kg > 0:
        cargas[importacion]['llegadas'][int(fecha.strftime('%Y%m%d'))] = cantidad_kg
    cargas[importacion]['fecha_final'] = int(fecha.strftime('%Y%m%d'))

    # Agregar las variables de inventario
    cargas[importacion]['inventario_al_final'] = dict()
    for t in periodos:
        var_name = f"O_{importacion}_{t}"
        lp_var = pu.LpVariable(name=var_name,
                               lowBound=0.0,
                               upBound=transitos_df.loc[i]['cantidad_kg'],
                               cat=pu.LpContinuous)
        cargas[importacion]['inventario_al_final'][t]=lp_var
    
# A Partir de los inventarios en puerto    
for i in inventario_puerto_df.index:
    empresa = inventario_puerto_df.loc[i]['empresa']
    operador = inventario_puerto_df.loc[i]['operador']
    puerto = inventario_puerto_df.loc[i]['puerto']
    ingrediente = inventario_puerto_df.loc[i]['ingrediente']
    importacion = str(inventario_puerto_df.loc[i]['importacion']).replace(' ','')
    inventario_inicial = inventario_puerto_df.loc[i]['cantidad_kg']
    valor_cif = inventario_puerto_df.loc[i]['valor_cif_kg']
    fecha = inventario_puerto_df.loc[i]['fecha_llegada']

    if not importacion in cargas.keys():
        cargas[importacion] = dict()
    
    cargas[importacion]['empresa'] = empresa
    cargas[importacion]['operador'] = operador
    cargas[importacion]['puerto'] = puerto
    cargas[importacion]['ingrediente'] = ingrediente
    cargas[importacion]['valor_cif'] = valor_cif
    cargas[importacion]['inventario_inicial'] = inventario_inicial
    cargas[importacion]['costo_almacenamiento'] = {int(t.strftime('%Y%m%d')):0 for t in fechas}

    # Poner llegadas de materia
    cargas[importacion]['llegadas'] = {t.strftime('%Y%m%d'):0 for t in fechas}
    
    cargas[importacion]['fecha_inicial'] = int(fecha.strftime('%Y%m%d'))
    cargas[importacion]['fecha_final'] = int(fecha.strftime('%Y%m%d'))
    # Agregar las variables de inventario
    cargas[importacion]['inventario_al_final'] = dict()

    for t in periodos:

        var_name = f"O_{importacion}_{t}"
        lp_var = pu.LpVariable(name=var_name,
                               lowBound=0.0,
                               upBound=inventario_puerto_df.loc[i]['cantidad_kg'],
                               cat=pu.LpContinuous)
        cargas[importacion]['inventario_al_final'][t]=lp_var
    

In [11]:
# Agregar costos de almacenamiento a cada carga
for i in costos_almacenamiento_df.index:
    importacion = costos_almacenamiento_df.loc[i]['importacion']
    fecha = int(costos_almacenamiento_df.loc[i]['fecha_corte'].strftime('%Y%m%d'))
    valor_kg = costos_almacenamiento_df.loc[i]['valor_kg']

    if importacion in cargas.keys():
        cargas[importacion]['costo_almacenamiento'][fecha] += valor_kg

In [12]:
# Agregar costos de bodegaje cuando es un producto en tránsito a puerto a cada carga
for importacion, carga in cargas.items():
    index = ('bodega', carga['operador'], carga['puerto'], carga['ingrediente'])
    valor_kg = operaciones_portuarias_df.loc[index]['valor_kg']
    if carga['fecha_inicial'] >= int(fechas[0].strftime('%Y%m%d')) and carga['fecha_final'] <= int(fechas[-1].strftime('%Y%m%d')):
        carga['costo_almacenamiento'][carga['fecha_final']] += valor_kg

In [13]:
intercompany_df = intercompany_df.melt(id_vars='origen',
                                       value_vars=['contegral', 'finca'],
                                       var_name='destino',
                                       value_name='intercompany')

intercompany_df.set_index(['origen', 'destino'], inplace=True)

In [14]:
# Encontrar el costo total de transporte por kilogramo
fletes_df = fletes_df.melt(id_vars=['puerto', 'operador', 'ingrediente'],
                           value_vars=list(plantas.keys()),
                           value_name='costo_per_kg',
                           var_name='planta')

# Calcular valor del flete
fletes_df['flete'] = cap_camion*fletes_df['costo_per_kg']

fletes_df = pd.merge(left=fletes_df,
                     right=plantas_df[['planta', 'empresa']],
                     left_on='planta',
                     right_on='planta')

fletes_df.set_index(['puerto', 'operador', 'ingrediente', 'planta'], inplace=True)

*Nota:* evitar crear variables de transporte si la capacidad de almacenamiento de la planta es 0 o e consumo es 0 

In [15]:
# Informacion de transporte
for importacion, carga in cargas.items():
    puerto = carga['puerto']
    operador = carga['operador']
    ingrediente = carga['ingrediente']
    costo_envio = dict() 

    for nombre_planta, planta in plantas.items():
        empresa_destino = planta['empresa']
        costo_intercompany = intercompany_df.loc[(carga['empresa'], empresa_destino)]['intercompany']
        valor_intercompany = cap_camion*carga['valor_cif']*(costo_intercompany)
        flete = fletes_df.loc[(puerto,operador,ingrediente,nombre_planta)]['flete']
        valor_despacho_directo_kg = cap_camion*operaciones_portuarias_df.loc[('directo', operador, puerto, ingrediente)]['valor_kg']

        # Costo de flete
        costo_envio[nombre_planta] = dict()
        costo_envio[nombre_planta]['intercompany'] = costo_intercompany
        costo_envio[nombre_planta]['flete'] = flete
        costo_envio[nombre_planta]['cantidad_despacho'] = cap_camion
        costo_envio[nombre_planta]['valor_intercompany'] = valor_intercompany
        costo_envio[nombre_planta]['costo_despacho_directo'] = valor_despacho_directo_kg

        costo_envio[nombre_planta]['costo_envio'] = dict()
        costo_envio[nombre_planta]['tipo_envio'] = dict()
        costo_envio[nombre_planta]['variable_despacho'] = dict()

        # Los transportes solo tienen sentido desde el periodo 3 y hasta leadtime dias antes del lead time
        periodo_inicial = 3
        lead_time = 2
        periodo_final = periodos.index(periodos[-1])-lead_time+1

        for periodo in periodos[3:periodo_final]:
            # Si el periodo esta entre la fecha de llegada, colocar operacion portuaria por despacho directo.
            if periodo >= carga['fecha_inicial'] and periodo <= carga['fecha_final']:
                costo_envio[nombre_planta]['costo_envio'][periodo] = valor_intercompany + flete + valor_despacho_directo_kg
                costo_envio[nombre_planta]['tipo_envio'][periodo] = 'directo'
                
            else:
                costo_envio[nombre_planta]['costo_envio'][periodo] = valor_intercompany + flete
                costo_envio[nombre_planta]['tipo_envio'][periodo] = 'indirecto'

            # Variable de transporte
                
            # Antes de crear las variables de transporte, es importante saber si la planta tiene consumo del ingrediente
            if ingrediente in planta['inventarios'].keys():            

                consumo_total = sum([c for p,c in planta['inventarios'][ingrediente]['consumo'].items()])

                if consumo_total > 0:

                    transporte_var_name = f'T_{importacion}_{nombre_planta}_{periodo}'
                    transporte_var = pu.LpVariable(name=transporte_var_name,
                                        lowBound=0,
                                        cat=pu.LpInteger)
                    
                    costo_envio[nombre_planta]['variable_despacho'][periodo] = transporte_var

                    # Colocar la variable en la planta dos periodos despues
                    periodo_llegada = periodos[periodos.index(periodo)+lead_time]
                    plantas[nombre_planta]['inventarios'][ingrediente]['llegadas'][periodo_llegada].append(transporte_var)           


        carga['costo_despacho'] = costo_envio
        

## Sets:

$t$: periodos con $t \in T$

$p$: productos con $p \in P$

$i$: cargas con $i \in T$

$j$: plantas con $j \in J$


## Parametros:

$CB:$ Costo de backorder por día

$CT_{ij}$: Costo de transportar la carga $i$ hacia la planta $j$

$CA_{it}$: Costo de mantener la carga $i$ almacenada al final del periodo $p$

$AR_{it}$: Cantidad de producto llegando a la carga $i$ durante el periodo $p$

$DM_{pjt}$: Demanda del producto $p$ en la planta $j$ durante el periodo $t$

$IP_{i}$: Inventario inicial de la carga $i$

$TJ_{pjt}$: Cantidad programada del producto $p$ llegando a la planta $j$ durante el periodo $t$ 

$IJ_{pj}$: Inventario inicial de producto $p$ en la planta $j$ 

$MI_{pjt}$: Inventario mínimo a mantener del producto $p$ en la planta $j$ al final del periodo $t$

$MX_{pjt}$: Inventario máximo a mantener del producto $p$ en la planta $j$ al final del periodo $t$


## Diccionarios

Carga[i][Producto]: Producto que contiene la carga i

Carga[i][Inventario_Inicial]: Inventario Inicial de la carga i
 


## Variables:

$T_{ijt}$: Variable entera, Cantidad de camiones de 34 Toneladas a despachar de la carga $i$ hasta la planta $j$ durante el periodo $t$

$O_{it}$: Continua, Cantidad de toneladas de la carga $i$ almacenadas al final del periodo $t$

$I_{pjt}$: Cantidad del producto $p$ almacenado en la planta $j$ al final del periodo $t$

$B_{pjt}$: Cantidad del producto $p$ de backorder en la planta $j$ al final del periodo $t$

$M_{pjt}$: Unidades de exceso del producto $p$ en la planta $j$ al final del periodo $t$ sobre el inventario objetivo

$X_{pjt}$: Unidades por debajo del producto $p$ en la planta $j$ al final del periodo $t$ bajo el inventario objetivo



## Funcion Objetivo:

$$min \sum_{i}\sum_{j}\sum_{t}{CT_{ijt}*T_{ijt}} + \sum_{i}\sum_{t}CA_{it}O_{it} + \sum_{pjt}{CB*B}  + PP \sum_{M}\sum_{P}\sum_{J}{M_{mpj}} + PP \sum_{M}\sum_{P}\sum_{J}{X_{mpj}}$$

In [16]:
funcion_objetivo = list()

### Costo por transporte

El costo del transporte esta dado por el producto escalar entre los costos de envio, que ya incluyen fletes, costos porturarios y costos intercompany

$$\sum_{i}\sum_{j}\sum_{t}{CT_{ijt}*T_{ijt}}$$

In [17]:
for periodo in periodos:
    for impo, carga in cargas.items():
        for nombre_planta, planta in plantas.items():
            if periodo in carga['costo_despacho'][nombre_planta]['variable_despacho'].keys():
                costo_envio = carga['costo_despacho'][nombre_planta]['costo_envio'][periodo]
                var_envio = carga['costo_despacho'][nombre_planta]['variable_despacho'][periodo]
                # CT_ijt*T_ijt
                funcion_objetivo.append(costo_envio*var_envio)


$\sum_{i}\sum_{t}CA_{it}O_{it}$

In [18]:
for periodo in periodos:
    for impo, carga in cargas.items():
        costo_almaenamiento = carga['costo_almacenamiento'][periodo]
        inventario_al_final = carga['inventario_al_final'][periodo]
        if costo_almaenamiento > 0:
            funcion_objetivo.append(costo_almaenamiento*inventario_al_final)


$\sum_{pjt}{CB*B}$

In [19]:
for nombre_planta, planta in plantas.items():
    for nombre_ingrediente, ingrediente in planta['inventarios'].items():
        for periodo, var in ingrediente['backorder'].items():
            funcion_objetivo.append(costo_backorder_dia*var)
        

Pendiente las penalizaciones

## Restricciones:

### Balance de masa en cargas

El inventario al final del periodo es igual a:

- el inventario al final del periodo anterior;
- más las llegadas planeadas;
- menos los despachos hacia plantas

$$ O_{it} =  O_{i(t-1)} + AR_{it} - 34000\sum_{J}{T_{ijt}} \hspace{1cm} \forall i \in I, t \in T$$

In [20]:
rest_balance_masa_puerto = list()

for importacion, carga in cargas.items():
    for periodo in periodos:

        left = list()
        right = list()
        
        # Oit
        Oit = carga['inventario_al_final'][periodo]
        left.append(Oit)

        # Oi(t-1)
        if periodo == periodos[0]:
            Oitant = carga['inventario_inicial']
        else:
            t_anterior = periodos[periodos.index(periodo)-1]
            Oitant = carga['inventario_al_final'][t_anterior]
        right.append(Oitant)

        # ARit
        if periodo in carga['llegadas'].keys():
            ar = carga['llegadas'][periodo]
            right.append(ar)

        # - 34000*Sum(Tijt)
        for planta, despacho in carga['costo_despacho'].items():            
            if periodo in despacho['variable_despacho'].keys():
                var_despacho = despacho['variable_despacho'][periodo]
                left.append(cap_camion*var_despacho)
        
        
        name = f'balance_masa_{importacion}_al_final_de_{periodo}'
        rest = (pu.lpSum(left) == pu.lpSum(right), name)
        
        rest_balance_masa_puerto.append(rest)


### Balance de masa en plantas

El inventario en planta al final del periodo es igual a:

- el inventario al final del periodo anterior;
- más las llegadas ya programadas;
- más las llegadas planeadas;
- menos la demanda
- más el backorder, que compensa cuando el inventario más las llegadas no son suficientes

$$ I_{pjt} = I_{pj(t-1)} + TJ_{pjt} + \sum_{i}{T_{ij(t-2)}} -  DM_{pjt} + B_{pjt} \hspace{1cm} \forall p \in P, j \in J, t \in T$$


In [21]:
plantas['envigado']['inventarios']['frijol']['backorder']

{}

In [22]:
rest_balance_masa_planta =  list()
for nombre_planta, planta in plantas.items():
    for nombre_ingrediente, ingrediente in planta['inventarios'].items():
        for periodo in periodos:

            left = list()
            right = list()

            # Ipjt
            Ipjt = ingrediente['inventario_final'][periodo]
            left.append(Ipjt)

            # Ipj(t-1)
            if periodo == periodos[0]:
                Ipj_tanterior = ingrediente['inventario_final'][periodo_anterior]
            else:
                p_anterior = periodos[periodos.index(periodo)-1]
                Ipj_tanterior = ingrediente['inventario_final'][p_anterior]
                
            right.append(Ipj_tanterior)
                
            # + TJ


            # + Tijt
            if periodo in ingrediente['llegadas'].keys():
                for llegada_planeada_var in ingrediente['llegadas'][periodo]:
                    right.append(cap_camion*llegada_planeada_var)

            # - DMpjt
           
            if periodo in ingrediente['consumo'].keys():
                DMpjt = ingrediente['consumo'][periodo]
                left.append(DMpjt)
            
            # + Baclorder
            if periodo in ingrediente['backorder'].keys():
                bak_var = ingrediente['backorder'][periodo]
                right.append(bak_var)

            name = f'balance_planta_{nombre_planta}_de_{nombre_ingrediente}_al_final_de_{periodo}'
            rest = (pu.lpSum(left) == pu.lpSum(right), name)

            rest_balance_masa_planta.append(rest)

### Capacidad de recepción por planta

En una planta y un periodo en particular, la suma del producto entre el tiempo del ingrediente y la cantidad de camiones llegando no debe superar el tiempo total disponible en la planta

$$ \sum_{I}{TiempoIngrediente_{pj}*T_{ijt}} \leq TiempoTotal \hspace{1cm} \forall p \in P, t \in T$$

### No superar el inventario máximo

El inventario al final de un día cualquiera debe estar bajo el nivel máximo, por lo que penalizaremos en la función objetivo una variable de holgura para tal efecto
$$ I_{pjt} \leq MX_{pjt} + M_{pjt} \hspace{1cm} \forall p \in P, j \in J, t \in T$$

### Superar el inventario mínimo

El inventario al final de un día cualquiera debe estar bajo el nivel máximo, por lo que penalizaremos en la función objetivo una variable de holgura para tal efecto
$$ I_{pjt} \geq MX_{pjt} + M_{pjt} \hspace{1cm} \forall p \in P, j \in J, t \in T$$

# Resolviendo el modelo

In [23]:
problema = pu.LpProblem(name='Bios_Solver', sense=pu.LpMinimize)

# Agregando funcion objetivo
problema+= pu.lpSum(funcion_objetivo)

# Agregando balance de masa puerto
for rest in rest_balance_masa_puerto:
    problema+=rest

In [24]:
for rest in rest_balance_masa_planta:
    problema+=rest

In [25]:
file.split('.')[0]

'model_template_1205v2'

In [26]:
t_limit_minutes = 5
cpu_count = max(1, os.cpu_count()-1)
gap = 5000000
print('cpu count', cpu_count)
engine = pu.PULP_CBC_CMD(
            timeLimit=60*t_limit_minutes,
            gapAbs=gap,
            warmStart=False,
            threads=cpu_count)
problema.solve(solver=engine)

cpu count 15
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/luispinilla/Documents/source_code/bios_solver/env/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/0p/md4rzs_n7bg992nr5mzyl7_00000gn/T/913b3d9b131044fe8dab488b9ab44a34-pulp.mps sec 300 allow 5000000 threads 15 timeMode elapsed branch printingOptions all solution /var/folders/0p/md4rzs_n7bg992nr5mzyl7_00000gn/T/913b3d9b131044fe8dab488b9ab44a34-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 4934 COLUMNS
At line 110281 RHS
At line 115211 BOUNDS
At line 135054 ENDATA
Problem MODEL has 4929 rows, 25670 columns and 48274 elements
Coin0008I MODEL read with 0 errors
seconds was changed from 1e+100 to 300
allowableGap was changed from 1e-10 to 5e+06
threads was changed from 0 to 15
Option for timeMode changed from cpu to elapsed
Continuous objective value is 2.68412e+12 - 0.07 seconds
Cgl0003I 0 fixed, 3 tightened bounds, 0 stre

1

In [58]:
for periodo, plan in cargas['0821CHANGMIN']['costo_despacho'].items():
    for planta, var in plan['variable_despacho'].items():
            if var.varValue > 0:
                print(periodo,planta, var, var.varValue)

cartago 20231205 T_0821CHANGMIN_cartago_20231205 13.0
cartago 20231206 T_0821CHANGMIN_cartago_20231206 19.0
cartago 20231207 T_0821CHANGMIN_cartago_20231207 14.0
cartago 20231209 T_0821CHANGMIN_cartago_20231209 11.0
cartago 20231210 T_0821CHANGMIN_cartago_20231210 11.0
cartago 20231211 T_0821CHANGMIN_cartago_20231211 12.0
cartago 20231212 T_0821CHANGMIN_cartago_20231212 13.0
cartago 20231213 T_0821CHANGMIN_cartago_20231213 9.0
launion 20231203 T_0821CHANGMIN_launion_20231203 16.0
launion 20231204 T_0821CHANGMIN_launion_20231204 5.0
launion 20231208 T_0821CHANGMIN_launion_20231208 3.0
launion 20231213 T_0821CHANGMIN_launion_20231213 3.0


# Construccion de reporte

In [64]:
reporte_puerto_dict=dict()
reporte_transporte_dict=dict()
reporte_plantas_dict=dict()

## Reporte de puerto

In [86]:
reporte_puerto_dict['importacion'] = list()
reporte_puerto_dict['empresa'] = list()
reporte_puerto_dict['operador'] = list()
reporte_puerto_dict['puerto'] = list()
reporte_puerto_dict['ingrediente'] = list()
reporte_puerto_dict['valor_cif'] = list()
reporte_puerto_dict['periodo'] = list()
reporte_puerto_dict['costo_almacenamiento'] = list()
reporte_puerto_dict['llegadas'] = list()
reporte_puerto_dict['inventario'] = list()

for importacion, carga in cargas.items():

    reporte_puerto_dict['importacion'].append(importacion)
    reporte_puerto_dict['empresa'].append(carga['empresa'])
    reporte_puerto_dict['operador'].append(carga['operador'])
    reporte_puerto_dict['puerto'].append(carga['puerto'])
    reporte_puerto_dict['ingrediente'].append(carga['ingrediente'])
    reporte_puerto_dict['valor_cif'].append(carga['valor_cif'])
    reporte_puerto_dict['periodo'].append(periodo_anterior)
    reporte_puerto_dict['costo_almacenamiento'].append(0.0)
    reporte_puerto_dict['llegadas'].append(0)
    reporte_puerto_dict['inventario'].append(carga['inventario_inicial'])
    

    for periodo in periodos:
        reporte_puerto_dict['importacion'].append(importacion)
        reporte_puerto_dict['empresa'].append(carga['empresa'])
        reporte_puerto_dict['operador'].append(carga['operador'])
        reporte_puerto_dict['puerto'].append(carga['puerto'])
        reporte_puerto_dict['ingrediente'].append(carga['ingrediente'])
        reporte_puerto_dict['valor_cif'].append(carga['valor_cif'])
        reporte_puerto_dict['periodo'].append(periodo)
        reporte_puerto_dict['costo_almacenamiento'].append(carga['costo_almacenamiento'][periodo])
        if periodo in carga['llegadas'].keys():
            reporte_puerto_dict['llegadas'].append(carga['llegadas'][periodo])
        else:
            reporte_puerto_dict['llegadas'].append(0.0)
        reporte_puerto_dict['inventario'].append(cargas['DEVBULKDENIZ']['inventario_al_final'][periodo].varValue)

reporte_puerto_df = pd.DataFrame(reporte_puerto_dict)

## Reporte transporte

In [133]:
reporte_transporte_dict['importacion'] = list()
reporte_transporte_dict['empresa'] = list()
reporte_transporte_dict['operador'] = list()
reporte_transporte_dict['puerto'] = list()
reporte_transporte_dict['ingrediente'] = list()
reporte_transporte_dict['periodo'] = list()
reporte_transporte_dict['tipo'] = list()
reporte_transporte_dict['planta'] = list()
reporte_transporte_dict['intercompany'] = list()
reporte_transporte_dict['costo_intercompany'] = list()
reporte_transporte_dict['flete'] = list()
reporte_transporte_dict['cantidad_despacho_por_camion'] = list()
reporte_transporte_dict['costo_portuario_despacho_directo'] = list()
reporte_transporte_dict['cantidad_despacho'] = list()
reporte_transporte_dict['cantidad_camiones_despachados'] = list()
reporte_transporte_dict['cantidad_despachada'] = list()
reporte_transporte_dict['costo_total_despacho'] = list()

for importacion, carga in cargas.items():
    for nombre_planta, despacho in carga['costo_despacho'].items():
        for periodo in periodos:
            if periodo in despacho['variable_despacho'].keys():
                if despacho['variable_despacho'][periodo].varValue > 0:

                    reporte_transporte_dict['importacion'].append(importacion)
                    reporte_transporte_dict['empresa'].append(carga['empresa'])
                    reporte_transporte_dict['operador'].append(carga['operador'])
                    reporte_transporte_dict['puerto'].append(carga['puerto'])
                    reporte_transporte_dict['ingrediente'].append(carga['ingrediente'])
                    reporte_transporte_dict['periodo'].append(periodo)
                    reporte_transporte_dict['tipo'].append(despacho['tipo_envio'][periodo])
                    reporte_transporte_dict['planta'].append(nombre_planta)
                    reporte_transporte_dict['intercompany'].append(despacho['intercompany'])
                    reporte_transporte_dict['costo_intercompany'].append(despacho['valor_intercompany'])
                    reporte_transporte_dict['flete'].append(despacho['flete'])
                    reporte_transporte_dict['cantidad_despacho_por_camion'].append(despacho['cantidad_despacho'])
                    reporte_transporte_dict['costo_portuario_despacho_directo'].append(despacho['costo_despacho_directo'])
                    reporte_transporte_dict['costo_total_despacho'].append(despacho['costo_envio'][periodo])
                    reporte_transporte_dict['cantidad_despacho'].append(cap_camion)
                    reporte_transporte_dict['cantidad_camiones_despachados'].append(despacho['variable_despacho'][periodo].varValue)
                    reporte_transporte_dict['cantidad_despachada'].append(cap_camion*despacho['variable_despacho'][periodo].varValue)

reporte_transporte_df = pd.DataFrame(reporte_transporte_dict)

## Reporte de Planta

In [135]:
reporte_plantas_dict=dict()
reporte_plantas_dict['planta'] = list()
reporte_plantas_dict['empresa'] = list()
reporte_plantas_dict['ingrediente'] = list()
reporte_plantas_dict['periodo'] = list()
reporte_plantas_dict['capcidad'] = list()
reporte_plantas_dict['consumo'] = list()
reporte_plantas_dict['backorder'] = list()
reporte_plantas_dict['inventario_final'] = list()

for nombre_planta, planta in plantas.items():
    for ingrediente, inventario in planta['inventarios'].items():

        reporte_plantas_dict['planta'].append(nombre_planta)
        reporte_plantas_dict['empresa'].append(planta['empresa'])
        reporte_plantas_dict['ingrediente'].append(ingrediente)
        reporte_plantas_dict['periodo'].append(periodo)
        reporte_plantas_dict['capcidad'].append(inventario['capacidad'])
        reporte_plantas_dict['consumo'].append(inventario['consumo'][periodo])
        reporte_plantas_dict['backorder'].append(0.0)
        reporte_plantas_dict['inventario_final'].append(inventario['inventario_final'][periodo_anterior])

        for periodo in periodos:
            reporte_plantas_dict['planta'].append(nombre_planta)
            reporte_plantas_dict['empresa'].append(planta['empresa'])
            reporte_plantas_dict['ingrediente'].append(ingrediente)
            reporte_plantas_dict['periodo'].append(periodo)
            reporte_plantas_dict['capcidad'].append(inventario['capacidad'])
            reporte_plantas_dict['consumo'].append(inventario['consumo'][periodo])
            if periodo in inventario['backorder'].keys():
                reporte_plantas_dict['backorder'].append(inventario['backorder'][periodo].varValue)
            else:
                reporte_plantas_dict['backorder'].append(0.0)
            
            if type(inventario['inventario_final'][periodo])==pu.pulp.LpVariable:
                reporte_plantas_dict['inventario_final'].append(inventario['inventario_final'][periodo].varValue)
            else:
                reporte_plantas_dict['inventario_final'].append(inventario['inventario_final'][periodo])

reporte_plantas_df = pd.DataFrame(reporte_plantas_dict)

In [136]:
with pd.ExcelWriter(path='archivo_schema.xlsx') as writer:
    reporte_puerto_df.to_excel(writer, sheet_name='Puertos', index=False)
    reporte_plantas_df.to_excel(writer, sheet_name='Plantas', index=False)
    reporte_transporte_df.to_excel(writer, sheet_name='Despachos', index=False)